In [ ]:
from ver_demo import Demo
from knowledgerepr import fieldnetwork
from modelstore.elasticstore import StoreHandler
from algebra import API
from ver_utils.column_selection import ColumnSelection
from ver_utils.query_by_example import ExampleColumn, QueryByExample
from ver_utils.materializer import Materializer

In [ ]:
graph_path = '/home/cc/advw_graph_02/'
store_client = StoreHandler()
network = fieldnetwork.deserialize_network(graph_path)
aurum_api = API(network=network, store_client=store_client)
demo = Demo(aurum_api)

In [ ]:
demo.qbe_interface()

In [ ]:
column_selection = ColumnSelection(aurum_api)
candidate_columns = column_selection.column_retreival(attr="", examples=['USD', 'CNY'])
for col in candidate_columns.values():
    print(col.tbl_name, col.attr_name, col.examples_set)

In [ ]:
example_columns = [ExampleColumn(attr='', examples=['United States', 'China']), ExampleColumn(attr='', examples=['USD', 'CNY'])]
qbe = QueryByExample(aurum_api)
candidate_list = qbe.find_candidate_columns(example_columns)
for candidate in candidate_list:
    print('00000')
    for x in candidate:
        print(x.tbl_name, x.attr_name)

In [ ]:
join_paths = qbe.find_joins_between_candidate_columns(candidate_list)

In [ ]:
for join_path in join_paths:
    print(join_path.to_str())

In [ ]:
materializer = Materializer(table_path='/home/cc/adventureWork/', sample_size=1000)
materializer.materialize(join_paths[1])

In [ ]:
import ddapi
from api.apiutils import DRS, Relation
hit = aurum_api._nid_to_hit('1357802409')
print(hit)
dd_api = ddapi.DDAPI(network)
drs = dd_api.drs_from_hit(hit)
res = aurum_api.neighbors(hit, Relation.CONTENT_SIM)
for x in res:
    print(x)